In [ ]:
#!/usr/bin/python3
# coding: utf-8
# Tokyo

In [ ]:
import codecs
from datetime import datetime as dt
from datetime import timedelta as td
import json
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    show_and_save_plotly

In [ ]:
def json2dict(filename, k, v):
    dic = {}
    with codecs.open(filename, encoding='utf-8') as f:
        jsn = json.load(f)
        print("{} 更新日: {}".format(filename, jsn['date']))
        npa = np.asarray(jsn['datasets']['data'])
        for l in npa:
            if l and l[k] and l[k] > 1:
                dic[l[k]] = int(l[v])
    return dic

In [ ]:
patient_dic = json2dict("patient.json", k='code', v='count')

In [ ]:
df_tokyo = pd.read_csv('tokyo.txt',header=0,
                 dtype={'code':int, 'patient':int, 'name':str, 'total':int, 'color':float})

for k, v in patient_dic.items():
    df_tokyo.at[df_tokyo['code'] == k, 'patient'] = v
    div = df_tokyo.loc[df_tokyo['code'] == k, 'total'].values
    df_tokyo.at[df_tokyo['code'] == k, 'color']  = (float(v) / div) * 100.0
#print(df)

In [ ]:
today_str = dt.now().isoformat()[:19].replace('T', ' ')
tw_body = "東京都 新型コロナ 全期間 罹患率(東京都 " + today_str + " 時点) "
f = codecs.open("tokyo-min.geojson", "r", encoding='utf-8')
filename = "tokyo-hm.jpg"
geojson = json.load(f)
f.close()
fig = px.choropleth(df_tokyo, geojson=geojson, color="color", hover_name='name',
                locations="code", featureidkey="properties.code",
                labels={'color':'値', 'code':'自治体コード'},
                projection="mercator", title=tw_body)
fig.update_geos(visible=False,
                lonaxis=dict(range=[138.967097, 139.921738]),
                lataxis=dict(range=[35.510499, 35.906622]))
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
show_and_save_plotly(fig, filename, js=False, show=True, image=True, html=True)

In [ ]:
tw = get_twitter()
tw_body += " https://geneasyura.github.io/cov19-hm/tokyo.html "
tweet_with_image(tw, "docs/images/{}".format(filename), tw_body)

In [ ]:
# 北部保健所管内における市を除く村町
hokubu_towns = ["国頭村", "大宜味村", "東村", "今帰仁村", "本部町", "伊江村", "伊平屋村", "伊是名村"]
hokubu_total = 39770
# 中部保健所管内における市を除く村町
chubu_towns = ["恩納村", "宜野座村", "金武町", "読谷村", "嘉手納町", "北谷町", "北中城村", "中城村"]
chubu_total = 144580
# 南部保健所管内における市を除く村町
nambu_towns = ["西原町", "与那原町", "南風原町", "八重瀬町", "渡嘉敷村", "座間味村", "粟国村", 
    "渡名喜村", "南大東村", "北大東村", "久米島町"]
nambu_total = 131988
# 八重山保健所管内における市を除く村町
yaeyama_towns = ["竹富町", "与那国町"]
yaeyama_total = 5841

In [ ]:
url = "https://raw.githubusercontent.com/Code-for-OKINAWA/covid19/master/data/"
download_if_needed(url, "data.json", "okinawa.json")

In [ ]:
def make_okinawa_hm(tw, days_after=0, title="沖縄県 新型コロナ 全期間 罹患率",
                   filename="okinawa-hm.jpg"):
    df_okinawa = pd.read_csv(
        "okinawa.txt", header=0,
        dtype={'code':int, 'name':str, 'total':int, 'color':float, 'vlaue':float})
    f = codecs.open("okinawa.min.geojson", "r", encoding='utf-8')
    geojson = json.load(f)
    f.close()
    
    f = codecs.open("okinawa.json", "rb", encoding="utf8")
    j = json.load(f)
    f.close()
    last_modified = j['patients']['date']
    print("最終更新：{}".format(last_modified))
    
    okinawa_sub = {}
    data = j['patients']['data']
    okinawa_sub['県外'] = 0
    for p in data:
        kakutei = dt.strptime(p['date'], "%Y-%m-%d") 
        one_week_ago = (dt.now() - td(days=days_after))
        can_count = (days_after == 0) or (days_after > 0 and kakutei >= one_week_ago)
        if can_count:
            loc = p['居住地']
            if loc.endswith('県') or loc.endswith('府') or loc.endswith('道') or loc.endswith('都'):
                okinawa_sub['県外'] += 1
            elif loc in okinawa_sub:
                okinawa_sub[loc] += 1
            else:
                okinawa_sub[loc] = 1
        else:
            pass
    print({k: v for k, v in sorted(okinawa_sub.items(), key=lambda i: i[1], reverse=True)})

    for index, row in df_okinawa.iterrows():
        nm = df_okinawa.loc[index, 'name']
        if nm in okinawa_sub:
            df_okinawa.at[index, 'value'] = okinawa_sub[nm]
            df_okinawa.at[index, 'color'] = (okinawa_sub[nm] / df_okinawa.loc[index, 'total']) * 100.0
        elif nm in hokubu_towns:
            df_okinawa.at[index, 'value'] = okinawa_sub['北部保健所管内']
            df_okinawa.at[index, 'color'] = (okinawa_sub['北部保健所管内'] / hokubu_total) * 100.0
        elif nm in chubu_towns:
            df_okinawa.at[index, 'value'] = okinawa_sub['中部保健所管内']
            df_okinawa.at[index, 'color'] = (okinawa_sub['中部保健所管内'] / chubu_total) * 100.0
        elif nm in nambu_towns:
            df_okinawa.at[index, 'value'] = okinawa_sub['南部保健所管内']
            df_okinawa.at[index, 'color'] = (okinawa_sub['南部保健所管内'] / nambu_total) * 100.0
        elif nm in yaeyama_towns:
            df_okinawa.at[index, 'value'] = okinawa_sub['八重山保健所管内']
            df_okinawa.at[index, 'color'] = (okinawa_sub['八重山保健所管内'] / yaeyama_total) * 100.0
    #print(df_okinawa)
    today_str = dt.now().isoformat()[:19].replace('T', ' ')
    tw_body = title + "(" + today_str + "時点) "
    fig = px.choropleth(df_okinawa, geojson=geojson, color="color", hover_name='name',
                    locations="code", featureidkey="properties.N03_007",
                    labels={'color':'値', 'code':'自治体コード'},
                    projection="mercator", title=tw_body)
    fig.update_geos(visible=False,
                    lonaxis=dict(range=[122.933138, 128.334120]),
                    lataxis=dict(range=[24.040284, 27.100481]))
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    show_and_save_plotly(fig, filename, js=False, show=True, image=True, html=True)    
    tw_body += " https://geneasyura.github.io/cov19-hm/okinawa.html "
    tweet_with_image(tw, "docs/images/{}".format(filename), tw_body)

In [ ]:
make_okinawa_hm(tw, days_after=0, title="沖縄県 新型コロナ 全期間 罹患率",
                filename="okinawa-hm.jpg")

In [ ]:
make_okinawa_hm(tw, days_after=7, title="沖縄県 新型コロナ 直近1週間 罹患率",
                filename="okinawa-hm-1w.jpg")

In [ ]:
url = "https://raw.githubusercontent.com/codeforosaka/covid19/master/data/"
download_if_needed(url, "data.json", "osaka.json")

In [ ]:
def make_osaka_hm(tw, days_after=0, title="大阪府 新型コロナ 全期間 罹患率",
                   filename="osaka-hm.jpg"):
    f = codecs.open("osaka.json", "rb", encoding="utf8")
    j = json.load(f)
    f.close()
    last_modified = j['patients']['date']
    print("最終更新：{}".format(last_modified))
    osaka_sub = {}
    data = j['patients']['data']
    one_week_ago = (dt.now() - td(days=days_after))
    for p in data:
        kakutei = dt.strptime(p['date'], "%Y-%m-%d") 
        can_count = (days_after == 0) or (days_after > 0 and kakutei >= one_week_ago)
        if can_count:
            loc = p['居住地']
            if loc in osaka_sub:
                osaka_sub[loc] += 1
            else:
                osaka_sub[loc] = 1
        else:
            pass
    print({k: v for k, v in sorted(osaka_sub.items(), key=lambda i: i[1], reverse=True)})    
    
    df_osaka = pd.read_csv(
        "osaka.txt", header=0,
        dtype={'code':int, 'name':str, 'total':int, 'color':float, 'vlaue':float})
    f = codecs.open("osaka.min.geojson", "r", encoding='utf-8')
    geojson = json.load(f)
    f.close()    
    
    osakashi_total = 2691185
    sakaishi_total = 839310    

    for index, row in df_osaka.iterrows():
        nm = df_osaka.loc[index, 'name']
        if nm in osaka_sub:
            df_osaka.at[index, 'value'] = osaka_sub[nm]
            df_osaka.at[index, 'color'] = (osaka_sub[nm] / df_osaka.loc[index, 'total']) * 100.0
        elif df_osaka.loc[index, 'code'] < 27140:
            df_osaka.at[index, 'value'] = osaka_sub["大阪市"]
            df_osaka.at[index, 'color'] = (osaka_sub["大阪市"] / osakashi_total) * 100.0        
        else:
            df_osaka.at[index, 'value'] = osaka_sub["堺市"]
            df_osaka.at[index, 'color'] = (osaka_sub["堺市"] / sakaishi_total) * 100.0    
    #print(df_osaka)
    today_str = dt.now().isoformat()[:19].replace('T', ' ')
    tw_body = title + "(" + today_str + "時点) "
    fig = px.choropleth(df_osaka, geojson=geojson, color="color", hover_name='name',
                    locations="code", featureidkey="properties.N03_007",
                    labels={'color':'値', 'code':'自治体コード'},
                    projection="mercator", title=tw_body)
    fig.update_geos(visible=False,
                    lonaxis=dict(range=[135.080065, 135.762591]),
                    lataxis=dict(range=[34.267691, 35.063983]))
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    show_and_save_plotly(fig, filename, js=False, show=True, image=True, html=True)
    tw_body += " https://geneasyura.github.io/cov19-hm/osaka.html "
    tweet_with_image(tw, "docs/images/{}".format(filename), tw_body)

In [ ]:
make_osaka_hm(tw, days_after=0, title="大阪府 新型コロナ 全期間 罹患率",
                   filename="osaka-hm.jpg")

In [ ]:
make_osaka_hm(tw, days_after=7, title="大阪府 新型コロナ 直近1週間 罹患率",
                   filename="osaka-hm-1w.jpg")

In [ ]:
url = "https://raw.githubusercontent.com/code4nagoya/covid19/master/data/"
download_if_needed(url, "patients.csv", "nagoya.csv")

In [ ]:
def make_nagoya_hm(tw, days_after=0, title="愛知県 新型コロナ 全期間 罹患率",
                   filename="nagoya-hm.jpg"):
    df_nagoya = pd.read_csv("nagoya.csv", header=0)
    
    df_nagoya['発表日'] = pd.to_datetime(df_nagoya['発表日'], format="%Y/%m/%d %H:%M")
    #print(df_nagoya)
    if days_after > 0:
        one_week_ago = (dt.now() - td(days=days_after))
        df_nagoya = df_nagoya[df_nagoya['発表日'] >= one_week_ago]
        #print(df_nagoya)
        
    nagoya_sub = {}
    nagoya_sub['県外'] = 0
    nagoya = df_nagoya.groupby('住居地').size()
    for i in nagoya.index:
        if  i.endswith('県') or  i.endswith('府') or  i.endswith('道') or \
            i.endswith('都') or i == "海外" or i == "中国武漢市":
            nagoya_sub['県外'] += 1
        else:    
            nagoya_sub[i] = nagoya[i]
    print({k: v for k, v in sorted(nagoya_sub.items(), key=lambda i: i[1], reverse=True)})
    
    df_nagoya = pd.read_csv(
        "nagoya.txt", header=0,
        dtype={'code':int, 'name':str, 'total':int, 'color':float, 'vlaue':float})
    f = codecs.open("nagoya.min.geojson", "r", encoding='utf-8')
    geojson = json.load(f)
    f.close()  
    
    nagoyashi_total = 2295638
    for index, row in df_nagoya.iterrows():
        nm = df_nagoya.loc[index, 'name']
        if nm in nagoya_sub:
            df_nagoya.at[index, 'value'] = nagoya_sub[nm]
            df_nagoya.at[index, 'color'] = (nagoya_sub[nm] / df_nagoya.loc[index, 'total']) * 100.0
        elif df_nagoya.loc[index, 'code'] < 23201:
            df_nagoya.at[index, 'value'] = nagoya_sub["名古屋市"]
            df_nagoya.at[index, 'color'] = (nagoya_sub["名古屋市"] / nagoyashi_total) * 100.0
    #print(df_nagoya)
    today_str = dt.now().isoformat()[:19].replace('T', ' ')
    tw_body = title + "(" + today_str + "時点) "
    fig = px.choropleth(df_nagoya, geojson=geojson, color="color", hover_name='name',
                    locations="code", featureidkey="properties.N03_007",
                    labels={'color':'値', 'code':'自治体コード'},
                    projection="mercator", title=tw_body)
    fig.update_geos(visible=False,
                    lonaxis=dict(range=[136.662557, 137.862727]),
                    lataxis=dict(range=[34.587322, 35.443081]))
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    show_and_save_plotly(fig, filename, js=False, show=True, image=True, html=True)
    tw_body += " https://geneasyura.github.io/cov19-hm/nagoya.html "
    tweet_with_image(tw, "docs/images/{}".format(filename), tw_body)

In [ ]:
make_nagoya_hm(tw, days_after=0, title="愛知県 新型コロナ 全期間 罹患率",
                   filename="nagoya-hm.jpg")

In [ ]:
# 名古屋はデータ反映少し遅れるため＋２日する
make_nagoya_hm(tw, days_after=7+2, title="愛知県 新型コロナ 直近1週間 罹患率",
                   filename="nagoya-hm-1w.jpg")

In [ ]:
fukuoka_guns = {
  'kasuya': {
     'total': 225585, 'name': "糟屋郡",
     'gun': ["宇美町", "篠栗町", "志免町", "須恵町", "新宮町", "久山町", "粕屋町"]
  },
  'onga': {
    'total': 93662, 'name': "遠賀郡",
    'gun': ["芦屋町", "水巻町", "岡垣町", "遠賀町"]
  },
  'kurate': {
    'total': 23817, 'name': "鞍手郡",
    'gun': ["小竹町", "鞍手町"]
  },
  'kaho': {
    'total': 13496, 'name': "嘉穂郡",
    'gun': ["桂川町"]
  },
  'asakura': {
    'total': 31480, 'name': "朝倉郡",
    'gun': ["筑前町", "東峰村"]
  },
  'mitsui': {
    'total': 15138, 'name': "三井郡",
    'gun': ["大刀洗町"]
  },
  'mizuma': {
    'total': 14176, 'name': "三潴郡",
    'gun': ["大木町"]
  },
  'yame': {
    'total': 20183, 'name': "八女郡",
    'gun': ["広川町"]
  },
  'tagawa': {
    'total': 77663, 'name': "田川郡",
    'gun': ["香春町", "添田町", "糸田町", "川崎町", "大任町", "赤村", "福智町"] 
  },
  'miyako': {
    'total': 55206, 'name': "京都郡",
    'gun': ["苅田町", "みやこ町"]
  },
  'chikujo': {
    'total': 32672, 'name': "築上郡",
    'gun': ["吉富町", "上毛町", "築上町"]
  }
}

In [ ]:
#for k, v in fukuoka_guns.items():
#    print(k, v['total'], v['name'], v['gun'])

In [ ]:
url = "https://ckan.open-governmentdata.org/dataset/8a9688c2-7b9f-4347-ad6e-de3b339ef740/resource/c27769a2-8634-47aa-9714-7e21c4038dd4/download/"
download_if_needed(url, "400009_pref_fukuoka_covid19_patients.csv", "fukuoka.csv")

In [ ]:
def make_fukuoka_hm(tw, days_after=0, title="福岡県 新型コロナ 全期間 罹患率",
                   filename="fukuoka-hm.jpg"):
    df_fukuoka = pd.read_csv("fukuoka.csv", header=0)
    df_fukuoka['公表_年月日'] = pd.to_datetime(df_fukuoka['公表_年月日'], format="%Y/%m/%d")
    #print(df_fukuoka)
    if days_after > 0:
        one_week_ago = (dt.now() - td(days=days_after))
        df_fukuoka = df_fukuoka[df_fukuoka['公表_年月日'] >= one_week_ago]
        #print(df_fukuoka)
    
    fukuoka_sub = {}
    fukuoka = df_fukuoka.groupby('居住地').size()

    fukuoka_sub['県外'] = 0
    for i in fukuoka.index:
        if  i.endswith('県') or  i.endswith('府') or  i.endswith('道') or \
            i.endswith('都') or i == "熊本市" or i.startswith("山口県") or \
            i == "大分市" or i == "海外" or i.startswith("佐賀県") or i.startswith("長崎県"):
            fukuoka_sub['県外'] += 1
        else:
            i_strip = i.replace('市内', '市')
            fukuoka_sub[i_strip] = fukuoka[i]
    print({k: v for k, v in sorted(fukuoka_sub.items(), key=lambda i: i[1], reverse=True)})
    df_fukuoka = pd.read_csv(
        "fukuoka.txt", header=0,
        dtype={'code':int, 'name':str, 'total':int, 'color':float, 'vlaue':float})
    f = codecs.open("fukuoka.min.geojson", "r", encoding='utf-8')
    geojson = json.load(f)
    f.close()    
    for index, row in df_fukuoka.iterrows():
        nm = df_fukuoka.loc[index, 'name']
        if nm in fukuoka_sub:
            if nm.startswith("北九州市"):
                w = round(fukuoka_sub["北九州市"] / 7.0)
            elif nm.startswith("福岡市"):
                w = round(fukuoka_sub["福岡市"] / 7.0)
            else:
                w = 0
            df_fukuoka.at[index, 'value'] = fukuoka_sub[nm] + w
            df_fukuoka.at[index, 'color'] = ((fukuoka_sub[nm] + w) / df_fukuoka.loc[index, 'total']) * 100.0
        else:
            for k, v in fukuoka_guns.items():
                if nm in v['gun'] and v['name'] in fukuoka_sub:
                    df_fukuoka.at[index, 'value'] = fukuoka_sub[v['name']] 
                    df_fukuoka.at[index, 'color'] = (fukuoka_sub[v['name']] / v['total']) * 100.0
                    break    
    #print(df_fukuoka)

    today_str = dt.now().isoformat()[:19].replace('T', ' ')
    tw_body = title + "(" + today_str + "時点) "
    fig = px.choropleth(df_fukuoka, geojson=geojson, color="color", hover_name='name',
                    locations="code", featureidkey="properties.N03_007",
                    labels={'color':'値', 'code':'自治体コード'},
                    projection="mercator", title=tw_body)
    fig.update_geos(visible=False,
                    lonaxis=dict(range=[129.992505, 131.195379]),
                    lataxis=dict(range=[32.999325, 33.936267]))
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    show_and_save_plotly(fig, filename, js=False, show=True, image=True, html=True)
    tw_body += " https://geneasyura.github.io/cov19-hm/fukuoka.html "
    tweet_with_image(tw, "docs/images/{}".format(filename), tw_body)

In [ ]:
make_fukuoka_hm(tw, days_after=0, title="福岡県 新型コロナ 全期間 罹患率",
                   filename="fukuoka-hm.jpg")

In [ ]:
make_fukuoka_hm(tw, days_after=7, title="福岡県 新型コロナ 直近1週間 罹患率",
                   filename="fukuoka-hm-1w.jpg")